In [ ]:
import dynamo as dyn


In [ ]:
print(dyn.__version__)
dyn.get_all_dependencies_version()

In [ ]:
# !pip install dynamo-release --upgrade --quiet

In [ ]:


import dynamo as dyn

# set white background
dyn.configuration.set_figure_params(background='white')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gseapy.plot import barplot, dotplot

import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.rc('legend',fontsize=32) # using a size in points

In [ ]:
import numpy as np
import pandas as pd
import loompy
import hdbscan
import matplotlib.pyplot as plt
import numba
import mygene
import scipy
from gseapy.plot import barplot, dotplot
import warnings
warnings.filterwarnings('ignore')

dyn.configuration.set_figure_params('dynamo', background='white')

## Specify inputs and outputs

In [ ]:
# specify inputs
main_folder = "/Users/kathleenabadie/Google Drive/1.Lab_starting_March_2018/5.Experiments_labwork/RNAseq/sci-FATE-seq/2019.06.11_scifate_expt1/2020Nova_Seq/RNA_velocity/"

full_RNA_folder = main_folder + "matrix_output_all_Tcell-cds/"
new_RNA_folder = main_folder + "matrix_output_new_Tcell-cds/"
df_cell_file = main_folder + "matrix_output_all_Tcell-cds/df_cell.tsv"

# specify outputs
output_folder = "/Users/kathleenabadie/Google Drive/1.Lab_starting_March_2018/5.Experiments_labwork/RNAseq/sci-FATE-seq/2019.06.11_scifate_expt1/202101_output/"
output_folder_new = "/Users/kathleenabadie/Google Drive/1.Lab_starting_March_2018/5.Experiments_labwork/RNAseq/sci-FATE-seq/2019.06.11_scifate_expt1/202108_output/"

# T cell umap coords
UMAP_coords_file_Tcell = output_folder + "df_cell_5k_Tcell_7clust.csv"

# here, specify names of columns in UMAP_coords_file to use for UMAP cooridnates and clusters for plotting
umap1 = 'umap_Tcell_5k_aligned_1'
umap2 = 'umap_Tcell_5k_aligned_2'
clusters = 'Tcell_5k_aligned_7clust'

# specify cluster colors
cluster_colors =["#008B8B", "#08519C","#969696", "#cc6541", "#A50F15","#3F007D", "#ab62c0"]

# TFs of interest from scifate modules
TF_Tcell = ['JUNB', 'MYC', 'TBX21', 'IRF4', 'EGR1', 'NFATC1', 'BMYC', 'MXI1', 'STAT5A', 'NFAT5', 'ELK3', 'EOMES', 'REL', 'BHLHE40', 'STAT3', 'RUNX2', 'FOXO3', 'MXD4', 'BCL11B', 'CUX1', 'GTF2I', 'FOXO1', 'FLI1', 'STAT1', 'CHD2', 'ZEB1', 'FOXN3', 'TCF7', 'LEF1', 'ELF1', 'MYB', 'IKZF1', 'TCF12']

# Import full list of linked genes (only necessary if subsetting adata to only include cell cycle or T cell module genes )
gene_mod_filename = '/Users/kathleenabadie/Google Drive/1.Lab_starting_March_2018/5.Experiments_labwork/RNAseq/sci-FATE-seq/2019.06.11_scifate_expt1/2020Nova_Seq/gene_mod_Tcell_all.txt'
gene_file = open(gene_mod_filename, 'r')
data = gene_file.readlines()

gene_mod = []
with open(gene_mod_filename) as my_file:
   for line in my_file:
       gene_mod.append(line.rstrip())
        


In [ ]:
# Import list of top 200 DEG between trajectories
DEG_filename = output_folder + "trajectory_terms_top200q.txt"
DEG_file = open(DEG_filename, 'r')
data = DEG_file.readlines()

DEG_bt_traj = []
with open(DEG_filename) as my_file:
   for line in my_file:
       DEG_bt_traj.append(line.rstrip())


## Data to anndata function

In [ ]:
def create_adata(input_folder_full, input_folder_new, gene_mod):
    #import full RNA data into adata structure
    adata = dyn.read_mtx(input_folder_full + 'gene_count.mtx').transpose()
    df_cell = pd.read_csv(input_folder_full + 'df_cell.tsv', delimiter="\t")
    df_gene = pd.read_csv(input_folder_full + 'df_gene.tsv', delimiter="\t")
    df_cell.index = df_cell["sample"]
    df_gene.index = df_gene["gene_id"]
    adata.obs = df_cell
    adata.var = df_gene
    adata_all = adata 
    
    #import new RNA data
    adata = dyn.read_mtx(input_folder_new + 'gene_count.mtx').transpose()
    df_cell = pd.read_csv(input_folder_new + 'df_cell.tsv', delimiter="\t")
    df_gene = pd.read_csv(input_folder_new + 'df_gene.tsv', delimiter="\t")
    df_cell.index = df_cell["sample"]
    df_gene.index = df_gene["gene_id"]
    adata.obs = df_cell
    adata.var = df_gene
    adata_new = adata
    
    #combine into a single adata structure
    # specify layers
    adata_all.layers['total'] = adata_all.X
    adata_all.layers['new'] = adata_new.X
    adata = adata_all   
    
    #import umap coordinates and other data from df_cell
    df_cell = pd.read_csv(UMAP_coords_file_Tcell)
    adata_filter = adata[df_cell["sample"], ]
    df_cell.index = df_cell["sample"]
    adata_filter.obs = df_cell
    adata_ori = adata
    adata = adata_filter
    # add label_time (2 hr)
    adata.obs['label_time'] = 2
    tkey = 'label_time'
    
    # Subset adata to only include T cell module genes
    adata = adata[:,adata.var['gene_short_name'].isin(gene_mod)]
    print(adata.var.shape)
    print(adata.shape)
    adata.var.head()
    
    # Change indexing from gene ID to gene name
    adata.var_names = adata.var['gene_short_name']

    # get cell cycle umap coordinates and cluster ID from df_cell and add to adata structure
    X_umap = df_cell[[umap1, umap2]]
    X_clusters = df_cell[[clusters]]
    adata.obsm['X_umap'] = X_umap.values 
    adata.obs['Clusters'] = X_clusters.values
    
    return(adata)

## Dynamo workflow function

In [ ]:
# Dynamo workflow
def dynamo_workflow(adata_in, spec_expt = False):
    # preprocess the data
    if (spec_expt == True):
        dyn.pp.recipe_monocle(adata_in, tkey = 'label_time', experiment_type='one_shot')
    else:
        dyn.pp.recipe_monocle(adata_in, tkey = 'label_time')
    #learn dynamics
    dyn.tl.dynamics(adata_in, NTR_vel = True)
    # dimension reduction (umap cooridinate have already been input to adata.obs, so this will be skipped) 
    dyn.tl.reduceDimension(adata_in)
    # calculate velocity
    dyn.tl.cell_velocities(adata_in, basis='umap')
    # Vector field reconstruction and characterization of topology
    dyn.vf.VectorField(adata_in, basis='umap')
    dyn.vf.topography(adata_in, basis='umap')
    

## Dynamo visualization function

In [ ]:
# Dynamo velocity visualization 
def dynamo_vis(adata_in, color_group, cluster_colors):
    dyn.pl.plot_energy(adata_in)
    dyn.pl.streamline_plot(adata_in,color=color_group, color_key = cluster_colors, show_legend=False, ncols=3, dpi=300, save_show_or_return='return')
    dyn.pl.basic_stats(adata_in)
    dyn.pl.show_fraction(adata_in)
#     dyn.pl.phase_portraits(adata, color = 'Clusters', discrete_continous_div_color_key = [cluster_colors_dict,cluster_colors_dict,cluster_colors_dict],genes=TF_Tcell)

## Run functions to create adata, run Dynamo workflow, and visualize
Do this first without specifying experiment type and next with specifying experiment type

In [ ]:
# Run without specifying experiment type
adata_test1 = create_adata(full_RNA_folder, new_RNA_folder, gene_mod)
dynamo_workflow(adata_test1, spec_expt=False)
dynamo_vis(adata_test1, color_group = 'Clusters', cluster_colors = cluster_colors)

## Test out differential geometry pipeline
Calculate RNA acceleration

In [ ]:
dyn.tl.cell_velocities(adata_test1, basis='pca')

In [ ]:
dyn.vf.VectorField(adata_test1,
                   basis='pca')

In [ ]:
dyn.vf.rank_velocity_genes(adata_test1,
                           groups='Clusters', 
                          vkey='velocity_N'); # Velocity_N is for labeling data (as opposed to splicing)


In [ ]:
rank_speed = adata_test1.uns['rank_velocity_N'];
rank_abs_speed = adata_test1.uns['rank_abs_velocity_N'];

In [ ]:
# save streamline plot
figure_name = "Tcell_newRNA_Tcell-mod-genes_20221004_noclusters"
dyn.pl.streamline_plot(adata_test1,color='Clusters', color_key = cluster_colors, show_legend=False, ncols=3, dpi=300, save_show_or_return='return')
# dyn.pl.streamline_plot(adata_test1, show_legend=False, ncols=3, dpi=300, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)

In [ ]:
adata_test1

In [ ]:
# plot expression for comparsion to velocity

dyn.pl.umap(adata_test1, color=['TCF7'], layer='X_total', frontier=False)
dyn.pl.umap(adata_test1, color=['TCF7'], layer='X_new', frontier=False)
dyn.pl.umap(adata_test1, color=['TCF7'], layer='velocity_N', frontier=False)

dyn.pl.umap(adata_test1, color=['FOXO1'], layer='X_total', frontier=False)
dyn.pl.umap(adata_test1, color=['FOXO1'], layer='X_new', frontier=False)
dyn.pl.umap(adata_test1, color=['FOXO1'], layer='velocity_N', frontier=False)

dyn.pl.umap(adata_test1, color=['ZEB1'], layer='X_total', frontier=False)
dyn.pl.umap(adata_test1, color=['ZEB1'], layer='X_new', frontier=False)
dyn.pl.umap(adata_test1, color=['ZEB1'], layer='velocity_N', frontier=False)

dyn.pl.umap(adata_test1, color=['BCL11B'], layer='X_total', frontier=False)
dyn.pl.umap(adata_test1, color=['BCL11B'], layer='X_new', frontier=False)
dyn.pl.umap(adata_test1, color=['BCL11B'], layer='velocity_N', frontier=False)

dyn.pl.umap(adata_test1, color=['EOMES'], layer='X_total', frontier=False)
dyn.pl.umap(adata_test1, color=['EOMES'], layer='X_new', frontier=False)
dyn.pl.umap(adata_test1, color=['EOMES'], layer='velocity_N', frontier=False)

In [ ]:
plt.rcParams.update({'font.size': 18, 
                    })

In [ ]:
figure_name = "Myc_vel_nostream"
dyn.pl.umap(adata_test1, color=['MYC'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)

In [ ]:
figure_name = "Irf4_vel_nostream"
dyn.pl.umap(adata_test1, color=['IRF4'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)

In [ ]:
figure_name = "Zeb2_vel_nostream"
dyn.pl.umap(adata_test1, color=['ZEB2'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)

In [ ]:
figure_name = "Foxo1_vel_nostream"
dyn.pl.umap(adata_test1, color=['FOXO1'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)

In [ ]:
figure_name = "Myb_vel_nostream"
dyn.pl.umap(adata_test1, color=['MYB'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)

In [ ]:
figure_name = "Zeb1_vel_nostream"
dyn.pl.umap(adata_test1, color=['ZEB1'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)

In [ ]:
figure_name = "Bcl11b_vel_nostream"
dyn.pl.umap(adata_test1, color=['BCL11B'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)

In [ ]:
figure_name = "Fli1_vel_nostream"
dyn.pl.umap(adata_test1, color=['FLI1'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)

In [ ]:
figure_name = "Egr1_vel_nostream"
dyn.pl.umap(adata_test1, color=['EGR1'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)

In [ ]:

figure_name = "Tcf7_vel_nostream"
dyn.pl.umap(adata_test1, color=['TCF7'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)
#dyn.pl.umap(adata_test1, color=['TCF7'], layer='acceleration', frontier=True)

In [ ]:
figure_name = "Myc_vel_nostream"
dyn.pl.umap(adata_test1, color=['MYC'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)


In [ ]:
figure_name = "Tbx21_vel_nostream"
dyn.pl.umap(adata_test1, color=['TBX21'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)
#dyn.pl.umap(adata_test1, color=['TBX21'], layer='acceleration', frontier=True)

In [ ]:
figure_name = "Runx2_vel"
dyn.pl.umap(adata_test1, color=['RUNX2'], layer='velocity_N', frontier=True, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)


In [ ]:
figure_name = "Zeb2_vel"
dyn.pl.umap(adata_test1, color=['ZEB2'], layer='velocity_N', frontier=True, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)


In [ ]:
figure_name = "Stat3_vel"
dyn.pl.umap(adata_test1, color=['STAT3'], layer='velocity_N', frontier=True, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)


In [ ]:
figure_name = "Mxi1_vel"
dyn.pl.umap(adata_test1, color=['MXI1'], layer='velocity_N', frontier=True, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)


In [ ]:
figure_name = "Eomes_vel_nostream"
dyn.pl.umap(adata_test1, color=['EOMES'], layer='velocity_N', frontier=False, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)


In [ ]:
figure_name = "Myb_vel"
dyn.pl.umap(adata_test1, color=['MYB'], layer='velocity_N', frontier=True, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)


In [ ]:
figure_name = "Stat5a_vel"
dyn.pl.umap(adata_test1, color=['STAT5A'], layer='velocity_N', frontier=True, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)


In [ ]:
figure_name = "BHLHE40_vel"
dyn.pl.umap(adata_test1, color=['BHLHE40'], layer='velocity_N', frontier=True, save_show_or_return='return')
dyn.pl.save_fig(path=output_folder_new+ figure_name, prefix=None, dpi=None, ext='pdf', transparent=True, close=True, verbose=True)
#dyn.pl.umap(adata_test1, color=['BHLHE40'], layer='acceleration', frontier=True)

In [ ]:
dyn.pl.umap(adata_test1, color=['MYB'], layer='velocity_N', frontier=True)
dyn.pl.umap(adata_test1, color=['MYB'], layer='acceleration', frontier=True)

In [ ]:
dyn.pl.umap(adata_test1, color=['EGR1'], layer='velocity_N', frontier=True)
dyn.pl.umap(adata_test1, color=['EGR1'], layer='acceleration', frontier=True)